In [14]:
#generate all code and results for the report in this file
import numpy as np 
import math 
from numba import njit
import random
def phospholipid_water_2D(L, N_total,water_fraction, head_radius_ratio,c_ratio, n_c_per_chain, area_fraction):
    #water_fraction is fraction of total particles that are water
    #head_radius_ratio is ratio of water particle radius to phospholipid head radius
    #c_ratio is ratio of water particle radius to carbon atom diameter in tail
    #area_fraction is total area fraction of all particles in system
    n_water = int(N_total * water_fraction)
    n_phospholipid = N_total - n_water
    n_head = n_phospholipid
    n_tail = n_phospholipid
    area_water = area_fraction * (L**2) / (n_water + head_radius_ratio**2 * n_head + (c_ratio**2 * n_c_per_chain * n_tail))
    r_water = (area_water / np.pi) ** 0.5
    r_head = head_radius_ratio * r_water
    r_carbon = c_ratio * r_water
    positions = [[],[],[]] 
    max_tries = 100000
    tries = 0
    # lists for heads, tails, water [list for tails will be a list with position of each carbon atom in tail - 2 lists per phosphate head]
    # ---- Insert phospholipids first ----
    # Insert heads at same time as tail
    while len(positions[0]) < n_head and tries < max_tries:
        tries += 1
        if tries % 1000 ==0:
            print(f'Tries = {tries}, Phospholipids placed = {len(positions[0])}/{n_head}')
        tail_positions = [] # xcoords and ycoords list for carbon atoms in tails
        head_pos = np.array([random.uniform(0, L), random.uniform(0, L)])
        angle = random.uniform(0, 2*np.pi)
        tail_x = (head_pos[0] + np.cos(angle)*(r_head + r_carbon))%L
        tail_y = (head_pos[1] + np.sin(angle)*(r_head + r_carbon))%L

        tail_positions.append(np.array([tail_x, tail_y]))
        while len(tail_positions) < n_c_per_chain:
            angle += np.random.normal(0, np.pi/16) # add some flexibility to tail correlates to real flexibility of phospholipid tails
            tail_x = (tail_positions[-1][0] + 2 * np.cos(angle)*r_carbon)%L
            tail_y = (tail_positions[-1][1] + 2* np.sin(angle)*r_carbon)%L
            tail_positions.append(np.array([tail_x, tail_y])) # tail positions is list of n carbon atoms lists with x and y coords
             #periodic boundary conditions
        #check for overlaps with existing particles
        if len(positions[0]) > 0:
            diffs_heads = np.array(positions[0]) - head_pos # check head - head overlap
            diffs_heads -= L * np.round(diffs_heads / L)
            dists_heads = np.linalg.norm(diffs_heads, axis=1)
            if not np.all(dists_heads >= 2 * r_head):
                continue
            #check head-tail overlaps
            overlap = False
            for existing_tail in positions[1]:
                diffs = np.array(existing_tail) - head_pos
                diffs -= L * np.round(diffs / L)
                if np.any(np.linalg.norm(diffs, axis=1) < (r_head + r_carbon)):
                    overlap = True
                    break
            if overlap:
                continue
            #check tail - head overlaps
            diffs_tail_head_total = []
            for tail_pos in tail_positions:
                diffs_tail_head = np.array(positions[0]) - np.array(tail_pos)
                diffs_tail_head -=L * np.round(diffs_tail_head / L)
                dists_tail_head = list(np.linalg.norm(diffs_tail_head, axis=1))
                diffs_tail_head_total.append(dists_tail_head)
            if not np.all(np.array(diffs_tail_head_total) >= r_head + r_carbon):
                continue
            #check tail - tail overlaps
            
            overlap = False
            for tail_pos in tail_positions:
                for existing_tail in positions[1]:
                    diffs = np.array(existing_tail) - tail_pos
                    diffs -= L * np.round(diffs / L)
                    if np.any(np.linalg.norm(diffs, axis=1) < 2 * r_carbon):
                        overlap = True
                        break
                if overlap:
                    break

            if overlap:
                continue
        positions[0].append(head_pos)
        positions[1].append(tail_positions)
    tries = 0
    #Now time to insert the water molecules
    while len(positions[2]) < n_water and tries < max_tries:
        water_pos = np.array([random.uniform(0, L), random.uniform(0, L)])
        tries += 1
        if tries % 1000 == 0:
            print(f'Tries for water = {tries}, Water placed = {len(positions[2])}/{n_water}')
        
        overlap = False
        #check for overlaps with all phospholipids
        diffs_water_head = np.array(positions[0]) - water_pos
        diffs_water_head -= L * np.round(diffs_water_head / L)
        dists_water_head = np.linalg.norm(diffs_water_head, axis=1)
        if not np.all(dists_water_head >= r_head + r_water):
            continue
        diffs_water_tail_total = []
        for existing_tail in positions[1]:
            diffs = np.array(existing_tail) - water_pos
            diffs -= L * np.round(diffs / L)
            if np.any(np.linalg.norm(diffs, axis=1) < (r_water + r_carbon)):
                overlap = True
                break
        if overlap:
            continue
        #check water - water overlaps
        if len(positions[2]) > 0:
            diffs_water_water = np.array(positions[2]) - water_pos
            diffs_water_water -= L * np.round(diffs_water_water / L)
            dists_water_water = np.linalg.norm(diffs_water_water, axis=1)
            if not np.all(dists_water_water >= 2 * r_water):
                continue
        positions[2].append(water_pos)
    
    
    return np.array(positions[0]), np.array(positions[1]), np.array(positions[2]), r_water, r_head, r_carbon

In [15]:


@njit
def monte_carlo_wp_2D(positions_heads, positions_tails, positions_water,
                      d_max, rotation_max, L, r_water, r_head, r_carbon,
                      n_c_per_chain):

    N_heads = positions_heads.shape[0]
    N_water = positions_water.shape[0]
    
    particle = int(np.random.rand() * (N_heads + N_water))

    displacement = d_max * np.random.normal()
    angle = 2.0 * math.pi * np.random.rand()
    x_displacement = displacement * np.cos(angle)
    y_displacement = displacement * np.sin(angle)

    rotation_angle =  math.pi/4 * (np.random.rand()-0.5)
    vec = positions_tails[particle, 0] - positions_heads[particle]
    initial_angle = np.arctan2(vec[1], vec[0])
    # ==========================================================
    # PHOSPHOLIPID MOVE
    # ==========================================================
    if particle < N_heads:

        moved_index = particle
        moved_type = 0

        pos_head = positions_heads[particle]

        # ---------- CHANGED: use NumPy array ----------
        new_pos_head = np.array([
            (pos_head[0] + x_displacement) % L,
            (pos_head[1] + y_displacement) % L
        ])

        # ---------- CHANGED: preallocate tail ----------
        new_tail_positions = np.empty((n_c_per_chain, 2))
        #need to find an initial angle so no preferential direction 

        new_tail_positions[0, 0] = (new_pos_head[0] + np.cos(rotation_angle+initial_angle) * (r_head + r_carbon)) % L
        new_tail_positions[0, 1] = (new_pos_head[1] + np.sin(rotation_angle+initial_angle) * (r_head + r_carbon)) % L

        for i in range(1, n_c_per_chain):

            prev_x = new_tail_positions[i-1, 0]
            prev_y = new_tail_positions[i-1, 1]

            if i == 1:
                ref_x, ref_y = new_pos_head
            else:
                ref_x = new_tail_positions[i-2, 0]
                ref_y = new_tail_positions[i-2, 1]

            initial_angle = math.atan2(prev_y - ref_y, prev_x - ref_x)
            new_angle = initial_angle + (rotation_max / 4.0) * np.random.normal()

            new_tail_positions[i, 0] = (prev_x + 2 * r_carbon * math.cos(new_angle)) % L
            new_tail_positions[i, 1] = (prev_y + 2 * r_carbon * math.sin(new_angle)) % L

        # ---------- overlap checks ----------

        # head–water
        for i in range(N_water):
            dx = positions_water[i, 0] - new_pos_head[0]
            dy = positions_water[i, 1] - new_pos_head[1]
            dx -= L * round(dx / L)
            dy -= L * round(dy / L)
            if dx*dx + dy*dy < (r_head + r_water)**2:
                return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        # head–head
        for j in range(N_heads):
            if j == particle:
                continue
            dx = positions_heads[j, 0] - new_pos_head[0]
            dy = positions_heads[j, 1] - new_pos_head[1]
            dx -= L * round(dx / L)
            dy -= L * round(dy / L)
            if dx*dx + dy*dy < (2 * r_head)**2:
                return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        # head–tail
        for j in range(N_heads):
            if j == particle:
                continue
            for k in range(n_c_per_chain):
                dx = positions_tails[j, k, 0] - new_pos_head[0]
                dy = positions_tails[j, k, 1] - new_pos_head[1]
                dx -= L * round(dx / L)
                dy -= L * round(dy / L)
                if dx*dx + dy*dy < (r_head + r_carbon)**2:
                    return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        # tail–water
        for k in range(n_c_per_chain):
            for i in range(N_water):
                dx = positions_water[i, 0] - new_tail_positions[k, 0]
                dy = positions_water[i, 1] - new_tail_positions[k, 1]
                dx -= L * round(dx / L)
                dy -= L * round(dy / L)
                if dx*dx + dy*dy < (r_water + r_carbon)**2:
                    return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        # tail–tail
        for j in range(N_heads):
            if j == particle:
                continue
            for k in range(n_c_per_chain):
                for m in range(n_c_per_chain):
                    dx = positions_tails[j, m, 0] - new_tail_positions[k, 0]
                    dy = positions_tails[j, m, 1] - new_tail_positions[k, 1]
                    dx -= L * round(dx / L)
                    dy -= L * round(dy / L)
                    if dx*dx + dy*dy < (2 * r_carbon)**2:
                        return positions_heads, positions_tails, positions_water, False, moved_index, moved_type
        #tail - heads
        for j in range(N_heads):
            for k in range(n_c_per_chain):
                dx = positions_heads[j, 0] - new_tail_positions[k, 0]
                dy = positions_heads[j, 1] - new_tail_positions[k, 1]
                dx -= L * round(dx / L)
                dy -= L * round(dy / L)
                if dx*dx + dy*dy < (r_head + r_carbon)**2:
                    return positions_heads, positions_tails, positions_water, False, moved_index, moved_type
        # ---------- accept ----------
        positions_heads[particle] = new_pos_head
        positions_tails[particle] = new_tail_positions

        return positions_heads, positions_tails, positions_water, True, moved_index, moved_type

    # ==========================================================
    # WATER MOVE (unchanged logic, array-safe)
    # ==========================================================
    else:
        particle -= N_heads
        moved_index = particle
        moved_type = 1

        pos_water = positions_water[particle]

        new_pos_water = np.array([
            (pos_water[0] + x_displacement) % L,
            (pos_water[1] + y_displacement) % L
        ])

        for i in range(N_water):
            if i == particle:
                continue
            dx = positions_water[i, 0] - new_pos_water[0]
            dy = positions_water[i, 1] - new_pos_water[1]
            dx -= L * round(dx / L)
            dy -= L * round(dy / L)
            if dx*dx + dy*dy < (2 * r_water)**2:
                return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        for j in range(N_heads):
            dx = positions_heads[j, 0] - new_pos_water[0]
            dy = positions_heads[j, 1] - new_pos_water[1]
            dx -= L * round(dx / L)
            dy -= L * round(dy / L)
            if dx*dx + dy*dy < (r_water + r_head)**2:
                return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

            for k in range(n_c_per_chain):
                dx = positions_tails[j, k, 0] - new_pos_water[0]
                dy = positions_tails[j, k, 1] - new_pos_water[1]
                dx -= L * round(dx / L)
                dy -= L * round(dy / L)
                if dx*dx + dy*dy < (r_water + r_carbon)**2:
                    return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        positions_water[particle] = new_pos_water
        return positions_heads, positions_tails, positions_water, True, moved_index, moved_type


In [16]:
@njit
def metropolis_pw_2D(positions_heads, positions_tails, positions_water, L, r_water, r_head, r_carbon, epsilon, Temperature, d_max, n_c_per_chain):
    """
    Metropolis algorithm for phospholipid system in 2D.
    Only nearest-neighbour interactions are considered.
    Assumes positions are numpy arrays:
    positions_heads: (N_heads, 2)
    positions_tails: (N_heads, n_c_per_chain, 2)
    positions_water: (N_water, 2)
    """
    N_heads = positions_heads.shape[0]
    N_water = positions_water.shape[0]
    
    # Make copies for trial moves
    pos_heads = np.copy(positions_heads)
    pos_tails = np.copy(positions_tails)
    pos_water = np.copy(positions_water)

    # Perform MC move (already njit)
    new_heads, new_tails, new_water, accepted, moved_index, moved_type = monte_carlo_wp_2D(
        positions_heads, positions_tails, positions_water, d_max, np.pi/8, 
        L, r_water, r_head, r_carbon, n_c_per_chain
    )

    if not accepted:
        return positions_heads, positions_tails, positions_water, False, moved_type

    # Initialize energies
    initial_energy = 0.0
    final_energy = 0.0

    # --- Head particle moved ---
    if moved_type == 0:
        # Tail-water interactions
        for c in range(n_c_per_chain):
            tail_old = pos_tails[moved_index, c]
            tail_new = new_tails[moved_index, c]

            # New
            diffs_new = new_water - tail_new
            diffs_new -= L * np.round(diffs_new / L)
            dists_new = np.sqrt(diffs_new[:,0]**2 + diffs_new[:,1]**2)
            for i in range(dists_new.shape[0]):
                if dists_new[i] < 1.75 * (r_carbon + r_water):
                    final_energy += epsilon

            # Old
            diffs_old = pos_water - tail_old
            diffs_old -= L * np.round(diffs_old / L)
            dists_old = np.sqrt(diffs_old[:,0]**2 + diffs_old[:,1]**2)
            for i in range(dists_old.shape[0]):
                if dists_old[i] < 1.75 * (r_carbon + r_water):
                    initial_energy += epsilon

        # Tail-tail interactions
        for c in range(n_c_per_chain):
            tail_new = new_tails[moved_index, c]
            tail_old = pos_tails[moved_index, c]

            for j in range(N_heads):
                if j == moved_index:
                    continue
                for cc in range(n_c_per_chain):
                    # New
                    diff = new_tails[j, cc] - tail_new
                    diff -= L * np.round(diff / L)
                    dist = np.sqrt(diff[0]**2 + diff[1]**2)
                    if dist < 1.75 * (r_carbon + r_carbon):
                        final_energy -= 1.2 * epsilon
                    # Old
                    diff_old = pos_tails[j, cc] - tail_old
                    diff_old -= L * np.round(diff_old / L)
                    dist_old = np.sqrt(diff_old[0]**2 + diff_old[1]**2)
                    if dist_old < 1.75 * (r_carbon + r_carbon):
                        initial_energy -= 1.2 * epsilon
        #head - water interactions
        diffs_new = new_water - new_heads[moved_index]
        diffs_new -= L * np.round(diffs_new / L)
        dists_new = np.sqrt(diffs_new[:,0]**2 + diffs_new[:,1]**2)
        for i in range(dists_new.shape[0]):
            if dists_new[i] < 1.75 * (r_head + r_water):
                final_energy += epsilon
        diffs_old = pos_water - pos_heads[moved_index]
        diffs_old -= L * np.round(diffs_old / L)
        dists_old = np.sqrt(diffs_old[:,0]**2 + diffs_old[:,1]**2)
        for i in range(dists_old.shape[0]):
            if dists_old[i] < 1.75 * (r_head + r_water):
                initial_energy += epsilon
    # --- Water particle moved ---
    else:
        water_old = pos_water[moved_index]
        water_new = new_water[moved_index]

        # Water-water interactions
        #for i in range(N_water):
            #if i == moved_index:
                #continue
            # New
            #diff = new_water[i] - water_new
            #diff -= L * np.round(diff / L)
            #dist = np.sqrt(diff[0]**2 + diff[1]**2)
            #if dist < 2 * (r_water + r_water):
                #final_energy -= epsilon
            # Old
            #diff_old = pos_water[i] - water_old
            #diff_old -= L * np.round(diff_old / L)
            #dist_old = np.sqrt(diff_old[0]**2 + diff_old[1]**2)
            #if dist_old < 2.5 * (r_water + r_water):
                #initial_energy -= epsilon

        # Water-tail interactions
        for i in range(N_heads):
            for c in range(n_c_per_chain):
                # New
                diff = new_tails[i, c] - water_new
                diff -= L * np.round(diff / L)
                dist = np.sqrt(diff[0]**2 + diff[1]**2)
                if dist < 1.75 * (r_carbon + r_water):
                    final_energy += epsilon
                # Old
                diff_old = pos_tails[i, c] - water_old
                diff_old -= L * np.round(diff_old / L)
                dist_old = np.sqrt(diff_old[0]**2 + diff_old[1]**2)
                if dist_old < 1.75 * (r_carbon + r_water):
                    initial_energy += epsilon

        #head - water interactions
        diffs_new = new_heads - water_new
        diffs_new -= L * np.round(diffs_new / L)
        dists_new = np.sqrt(diffs_new[:,0]**2 + diffs_new[:,1]**2)
        for i in range(dists_new.shape[0]):
            if dists_new[i] < 1.75 * (r_head + r_water):
                final_energy += epsilon
        diffs_old = pos_heads - water_old
        diffs_old -= L * np.round(diffs_old / L)
        dists_old = np.sqrt(diffs_old[:,0]**2 + diffs_old[:,1]**2)
        for i in range(dists_old.shape[0]):
            if dists_old[i] < 1.75 * (r_head + r_water):
                initial_energy += epsilon

    # Metropolis criterion
    delta_E = final_energy - initial_energy
    if delta_E < 0.0 or np.exp(-delta_E / (k_b * Temperature)) >= np.random.rand():
        return new_heads, new_tails, new_water, True, moved_type
    else:
        return pos_heads, pos_tails, pos_water, False, moved_type
    

In [17]:
def phospholipid_pair_correlation_2D(positions, r_water, r_head, r_carbon, L, bins):
    #positions is list positions = [heads, tails, water]
    #calculate pair correlation function for phospholipid heads and tails separately
    #only consider head - head interactions and tail - tail interactions for pair correlation function calculations
    #bins is number of bins for pair correlation function calculation
    #calculate g(r) for heads
    head_positions = np.array(positions[0])
    tail_positions = np.array(positions[1]).reshape(-1, 2) # reshape tail positions to be a list of all tail particle positions
    r_head_head, g_r_head_head = pair_correlation_2D(head_positions, r_head*2, L, bins)
    r_tail_tail, g_r_tail_tail = pair_correlation_2D(tail_positions, r_carbon*2, L, bins)
    return r_head_head, g_r_head_head, r_tail_tail, g_r_tail_tail

#plot the positions found above 

In [18]:
#define function to identify monomer clusters in phospholipid system based on tail-tail contacts
@njit
def identify_monomer_clusters(positions_heads, positions_tails,
                              L, r_carbon, n_c_per_chain):

    N = positions_heads.shape[0]

    # adjacency matrix
    connected = np.zeros((N, N), dtype=np.int32)

    cutoff_sq = (4 * r_carbon)**2

    # -----------------------------
    # Build connectivity matrix
    # -----------------------------
    for i in range(N):
        for j in range(i+1, N):

            is_connected = False

            # tail–tail contact check
            for c1 in range(n_c_per_chain):
                for c2 in range(n_c_per_chain):

                    dx = positions_tails[i, c1, 0] - positions_tails[j, c2, 0]
                    dy = positions_tails[i, c1, 1] - positions_tails[j, c2, 1]

                    dx -= L * round(dx / L)
                    dy -= L * round(dy / L)

                    if dx*dx + dy*dy < cutoff_sq:
                        is_connected = True
                        break

                if is_connected:
                    break

            if is_connected:
                connected[i, j] = 1
                connected[j, i] = 1

    # -----------------------------
    # Find connected components
    # -----------------------------
    visited = np.zeros(N, dtype=np.int32)
    monomer_count = 0

    for i in range(N):

        if visited[i] == 1:
            continue

        # DFS stack
        stack = [i]
        visited[i] = 1
        size = 1

        while len(stack) > 0:
            node = stack.pop()

            for j in range(N):
                if connected[node, j] == 1 and visited[j] == 0:
                    visited[j] = 1
                    stack.append(j)
                    size += 1

        # If component size == 1 → monomer
        if size == 1:
            monomer_count += 1

    return monomer_count / N


In [19]:
#define function that identifies micelle cluster sizes in phospholipid system based on tail-tail contacts
@njit
def average_micelle_size(positions_heads, positions_tails,
                         L, r_carbon, n_c_per_chain):

    N = positions_heads.shape[0]

    # small buffer for numerical stability
    buffer = 2 * r_carbon
    cutoff_sq = (2 * r_carbon + buffer)**2

    # adjacency matrix
    connected = np.zeros((N, N), dtype=np.int32)

    # -----------------------------
    # Build connectivity matrix
    # -----------------------------
    for i in range(N):
        for j in range(i+1, N):

            is_connected = False

            for c1 in range(n_c_per_chain):
                for c2 in range(n_c_per_chain):

                    dx = positions_tails[i, c1, 0] - positions_tails[j, c2, 0]
                    dy = positions_tails[i, c1, 1] - positions_tails[j, c2, 1]

                    dx -= L * round(dx / L)
                    dy -= L * round(dy / L)

                    if dx*dx + dy*dy < cutoff_sq:
                        is_connected = True
                        break

                if is_connected:
                    break

            if is_connected:
                connected[i, j] = 1
                connected[j, i] = 1

    # -----------------------------
    # Find connected components
    # -----------------------------
    visited = np.zeros(N, dtype=np.int32)

    total_micelle_size = 0
    micelle_count = 0

    for i in range(N):

        if visited[i] == 1:
            continue

        # DFS
        stack = [i]
        visited[i] = 1
        size = 1

        while len(stack) > 0:
            node = stack.pop()

            for j in range(N):
                if connected[node, j] == 1 and visited[j] == 0:
                    visited[j] = 1
                    stack.append(j)
                    size += 1

        # if cluster larger than monomer
        if size > 1:
            total_micelle_size += size
            micelle_count += 1

    if micelle_count == 0:
        return 0.0

    return total_micelle_size / micelle_count

@njit
def pbc_dist2(x1, y1, x2, y2, L):
    dx = x1 - x2
    dy = y1 - y2

    dx -= L * np.round(dx / L)
    dy -= L * np.round(dy / L)

    return dx*dx + dy*dy
# --------------------------------------------------
# Identify clusters using simple BFS
# --------------------------------------------------
@njit
def find_clusters(heads, L, r_cut):

    N = heads.shape[0]
    visited = np.zeros(N, dtype=np.int32)
    cluster_sizes = np.zeros(N, dtype=np.int32)

    r_cut2 = r_cut * r_cut
    n_clusters = 0

    for i in range(N):

        if visited[i] == 1:
            continue

        # Start new cluster
        stack = np.zeros(N, dtype=np.int32)
        stack_size = 0

        stack[stack_size] = i
        stack_size += 1
        visited[i] = 1

        size = 0

        while stack_size > 0:

            stack_size -= 1
            j = stack[stack_size]
            size += 1

            for k in range(N):
                if visited[k] == 0:
                    d2 = pbc_dist2(
                        heads[j,0], heads[j,1],
                        heads[k,0], heads[k,1], L
                    )

                    if d2 < r_cut2:
                        visited[k] = 1
                        stack[stack_size] = k
                        stack_size += 1

        cluster_sizes[n_clusters] = size
        n_clusters += 1

    return cluster_sizes[:n_clusters]

In [20]:
@njit
def micelle_statistics(cluster_sizes, N_total):

    # Maximum possible size is N_total
    H = np.zeros(N_total+1, dtype=np.int32)

    # Build histogram H_i
    for i in range(cluster_sizes.shape[0]):
        size = cluster_sizes[i]
        if size > 1:      # exclude monomers
            H[size] += 1

    numerator_Nn = 0.0
    denominator_Nn = 0.0

    numerator_Nw = 0.0
    denominator_Nw = 0.0

    for i in range(2, N_total+1):
        if H[i] > 0:
            numerator_Nn += i * H[i]
            denominator_Nn += H[i]

            numerator_Nw += i * i * H[i]
            denominator_Nw += i * H[i]

    if denominator_Nn == 0.0:
        return 0.0, 0.0

    Nn = numerator_Nn / denominator_Nn
    Nw = numerator_Nw / denominator_Nw

    return Nn, Nw
def compute_micelle_sizes(heads, L, r_cut):

    cluster_sizes = find_clusters(heads, L, r_cut)

    Nn, Nw = micelle_statistics(cluster_sizes, heads.shape[0])

    return Nn, Nw

In [31]:
#all code below from ai to compute the asphericities of micelles in my system
@njit
def minimum_image(dx, L):
    return dx - L * np.round(dx / L)
@njit
def identify_micelles(positions_heads, L, r_cut):
    N = positions_heads.shape[0]
    visited = np.zeros(N, dtype=np.uint8)
    cluster_id = -np.ones(N, dtype=np.int32)
    
    r_cut2 = r_cut * r_cut
    stack = np.empty(N, dtype=np.int32)
    
    current_cluster = 0
    
    for i in range(N):
        if visited[i] == 1:
            continue
        
        stack_size = 1
        stack[0] = i
        
        while stack_size > 0:
            j = stack[stack_size - 1]
            stack_size -= 1
            
            if visited[j] == 1:
                continue
            
            visited[j] = 1
            cluster_id[j] = current_cluster
            
            for k in range(N):
                if visited[k] == 0:
                    dx = minimum_image(
                        positions_heads[j,0] - positions_heads[k,0], L
                    )
                    dy = minimum_image(
                        positions_heads[j,1] - positions_heads[k,1], L
                    )
                    
                    if dx*dx + dy*dy < r_cut2:
                        stack[stack_size] = k
                        stack_size += 1
        
        current_cluster += 1
    
    return cluster_id, current_cluster

@njit
def compute_asphericities_full(
    positions_heads,
    positions_tails,
    cluster_id,
    n_clusters,
    L,
    min_size
):
    N = positions_heads.shape[0]
    n_tail = positions_tails.shape[1]
    
    A_values = np.zeros(n_clusters)
    cluster_sizes = np.zeros(n_clusters, dtype=np.int32)
    
    # Count lipid numbers per cluster
    for i in range(N):
        cid = cluster_id[i]
        if cid >= 0:
            cluster_sizes[cid] += 1
    
    for cid in range(n_clusters):
        
        if cluster_sizes[cid] < min_size:
            continue
        
        # ---- find reference lipid ----
        ref = -1
        for i in range(N):
            if cluster_id[i] == cid:
                ref = i
                break
        
        x_ref = positions_heads[ref,0]
        y_ref = positions_heads[ref,1]
        
        # ---- compute centre of mass over ALL beads ----
        total_beads = cluster_sizes[cid] * (1 + n_tail)
        
        x_sum = 0.0
        y_sum = 0.0
        
        for i in range(N):
            if cluster_id[i] == cid:
                
                # --- head ---
                dx = minimum_image(
                    positions_heads[i,0] - x_ref, L
                )
                dy = minimum_image(
                    positions_heads[i,1] - y_ref, L
                )
                
                x_head = x_ref + dx
                y_head = y_ref + dy
                
                x_sum += x_head
                y_sum += y_head
                
                # --- tail beads ---
                for t in range(n_tail):
                    
                    dx = minimum_image(
                        positions_tails[i,t,0] - x_ref, L
                    )
                    dy = minimum_image(
                        positions_tails[i,t,1] - y_ref, L
                    )
                    
                    x_tail = x_ref + dx
                    y_tail = y_ref + dy
                    
                    x_sum += x_tail
                    y_sum += y_tail
        
        x_cm = x_sum / total_beads
        y_cm = y_sum / total_beads
        
        # ---- build gyration tensor ----
        a = 0.0
        b = 0.0
        c = 0.0
        
        for i in range(N):
            if cluster_id[i] == cid:
                
                # head
                dx = minimum_image(
                    positions_heads[i,0] - x_ref, L
                )
                dy = minimum_image(
                    positions_heads[i,1] - y_ref, L
                )
                
                x = x_ref + dx - x_cm
                y = y_ref + dy - y_cm
                
                a += x*x
                b += x*y
                c += y*y
                
                # tails
                for t in range(n_tail):
                    
                    dx = minimum_image(
                        positions_tails[i,t,0] - x_ref, L
                    )
                    dy = minimum_image(
                        positions_tails[i,t,1] - y_ref, L
                    )
                    
                    x = x_ref + dx - x_cm
                    y = y_ref + dy - y_cm
                    
                    a += x*x
                    b += x*y
                    c += y*y
        
        a /= total_beads
        b /= total_beads
        c /= total_beads
        
        trace = a + c
        diff  = a - c
        
        root = np.sqrt(0.25*diff*diff + b*b)
        
        lambda1 = 0.5*trace + root
        lambda2 = 0.5*trace - root
        
        if trace > 0.0:
            A_values[cid] = ((lambda1 - lambda2)**2) / (trace*trace)
    count = 0
    total = 0.0

    for cid in range(n_clusters):
        if cluster_sizes[cid] >= min_size:
            total += A_values[cid]
            count += 1

    if count > 0:
        asphericity_av = total / count
    else:
        asphericity_av = 0.0

    return asphericity_av

@njit
def micelle_asphericity_system_full(
    positions_heads,
    positions_tails,
    L,
    r_cut,
    min_size
):
    cluster_id, n_clusters = identify_micelles(
        positions_heads, L, r_cut
    )
    
    return compute_asphericities_full(
        positions_heads,
        positions_tails, 
        cluster_id,
        n_clusters,
        L,
        min_size
    )

def rc_from_P(P, rh):
    return (4 * P / math.pi) * rh

In [ ]:
#now that we have defined all variables and functions needed for calculations, can begin generating the simulations needed for report
#want to investigate the monomer count, micelle averages for fixed Packing parameter - do for one high and one low 
#investigate the asphericity for different packing parameters therefore probably need about 10 different P with minimum 5 runs of each
#and only generating the asphericity parameter every 5000 runs after has run for about 1.5x10^7 - and collect then 100 data points which can be averaged for asphericity
#calculate standard error and evarge for each value of asphericity
#could also include plot of asphericity w.r.t monte carlo steps to show that after certain no. of steps there is only random change - chi-squared analysis to asses
#would show that the system has equillibrated and reached a minimum energy state
#now set the general parameters kept constant across all systems
L = 1
N = 500
eta = 0.7
head_radius_ratio = 1.2 #keep this constant and vary the ratio of the carbon atoms to this
area_fraction = 0.3
epsilon = 1
t = 1.8
k_b = 1.4 * 10 ** -23
Temperature = t * epsilon / k_b
n_c_per_tail = 6 #weirdly does not affect the packing parameter of the system - should show in the report why 
#d_max = r_water / 10 - can only set this once the radius of water has been defined   

In [23]:
# for first packing parameter calculations keep the number of carbon atoms in the tail constant 
P_values = [0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75] #10 values to evalute the asphericity for

In [26]:
#generate radius ratio values from the packing parameter 
c_ratios = []
for P_value in P_values:
    c_ratio = rc_from_P(P_value, head_radius_ratio)
    c_ratios.append(c_ratio)

#from list of Carbon tail ratios generate the initial configurations for all systems

h_03 , t_03, w_03, r_w_03, r_h_03, r_c_03 = phospholipid_water_2D(L, N, eta, head_radius_ratio, c_ratios[0], n_c_per_tail, area_fraction)
h_035, t_035, w_035, r_w_035, r_h_035, r_c_035 = phospholipid_water_2D(L,N, eta, head_radius_ratio, c_ratios[1],n_c_per_tail, area_fraction)
h_04, t_04, w_04, r_w_04, r_h_04, r_c_04 = phospholipid_water_2D(L, N, eta, head_radius_ratio, c_ratios[2], n_c_per_tail, area_fraction)
h_045, t_045, w_045, r_w_045, r_h_045, r_c_045 = phospholipid_water_2D(L, N, eta, head_radius_ratio,c_ratios[3],n_c_per_tail, area_fraction )
h_05, t_05, w_05, r_w_05, r_h_05, r_c_05 = phospholipid_water_2D(L, N, eta, head_radius_ratio,c_ratios[4],n_c_per_tail, area_fraction )
h_055, t_055, w_055, r_w_055, r_h_055, r_c_055 = phospholipid_water_2D(L, N, eta, head_radius_ratio,c_ratios[5],n_c_per_tail, area_fraction )
h_06, t_06, w_06, r_w_06, r_h_06, r_c_06 = phospholipid_water_2D(L, N, eta, head_radius_ratio,c_ratios[6],n_c_per_tail, area_fraction )
h_065, t_065, w_065, r_w_065, r_h_065, r_c_065 = phospholipid_water_2D(L, N, eta, head_radius_ratio,c_ratios[7],n_c_per_tail, area_fraction )
h_07, t_07, w_07, r_w_07, r_h_07, r_c_07 = phospholipid_water_2D(L, N, eta, head_radius_ratio,c_ratios[8],n_c_per_tail, area_fraction )
h_075, t_075, w_075, r_w_075, r_h_075, r_c_075 = phospholipid_water_2D(L, N, eta, head_radius_ratio,c_ratios[9],n_c_per_tail, area_fraction )

#10 initiail configurations with different packing parameters for the phospholipids only varying the size of the carbon atoms in each tail
d_max = r_w_04 / 10

Tries for water = 1000, Water placed = 284/350
Tries for water = 1000, Water placed = 286/350
Tries for water = 1000, Water placed = 304/350
Tries for water = 1000, Water placed = 293/350
Tries for water = 1000, Water placed = 335/350
Tries = 1000, Phospholipids placed = 144/150
Tries for water = 1000, Water placed = 294/350
Tries = 1000, Phospholipids placed = 144/150
Tries for water = 1000, Water placed = 319/350
Tries = 1000, Phospholipids placed = 146/150
Tries for water = 1000, Water placed = 345/350
Tries = 1000, Phospholipids placed = 145/150
Tries = 1000, Phospholipids placed = 144/150
Tries for water = 1000, Water placed = 348/350


In [ ]:
#now write the code to check the asphericity parameter measurement after some monte carlo moves in the lowest packing parameter system
#using P = 0.3
Nn_total_03 = []
Nw_total_03 = []
monomer_list_03 = []
asphericity_03 = []
for i in range(1000,15000000):
    h_03, t_03, w_03, accepted, moved_type = metropolis_pw_2D(h_03, t_03, w_03, L, r_w_03, r_h_03, r_c_03, epsilon, Temperature, d_max, n_c_per_tail)
    if i % 10000 == 0:
        monomer_count = identify_monomer_clusters(h_03, t_03, L, r_c_03, n_c_per_tail)
        monomer_list_03.append(monomer_count)
        Nn, Nw = compute_micelle_sizes(h_03,L,3.5*r_h_03)
        Nn_total_03.append(Nn)
        Nw_total_03.append(Nw)
        A_value = micelle_asphericity_system_full(h_03, t_03, L, 2.5 * r_h_03, 5)
        asphericity_03.append(A_value)
        print(f'Move {i/10000}')



Move 1.0
Move 2.0
Move 3.0
Move 4.0
Move 5.0
Move 6.0
Move 7.0
Move 8.0
Move 9.0
Move 10.0
Move 11.0
Move 12.0
Move 13.0
Move 14.0
Move 15.0
Move 16.0
Move 17.0
Move 18.0
Move 19.0
Move 20.0
Move 21.0
Move 22.0
Move 23.0
Move 24.0
Move 25.0
Move 26.0
Move 27.0
Move 28.0
Move 29.0
Move 30.0
Move 31.0
Move 32.0
Move 33.0
Move 34.0
Move 35.0
Move 36.0
Move 37.0
Move 38.0
Move 39.0
Move 40.0
Move 41.0
Move 42.0
Move 43.0
Move 44.0
Move 45.0
Move 46.0
Move 47.0
Move 48.0
Move 49.0
Move 50.0
Move 51.0
Move 52.0
Move 53.0
Move 54.0
Move 55.0
Move 56.0
Move 57.0
Move 58.0
Move 59.0
Move 60.0
Move 61.0
Move 62.0
Move 63.0
Move 64.0
Move 65.0
Move 66.0
Move 67.0
Move 68.0
Move 69.0
Move 70.0
Move 71.0
Move 72.0
Move 73.0
Move 74.0
Move 75.0
Move 76.0
Move 77.0
Move 78.0
Move 79.0
Move 80.0
Move 81.0
Move 82.0
Move 83.0
Move 84.0
Move 85.0
Move 86.0
Move 87.0
Move 88.0
Move 89.0
Move 90.0
Move 91.0
Move 92.0
Move 93.0
Move 94.0
Move 95.0
Move 96.0
Move 97.0
Move 98.0
Move 99.0
Move 100.0
Move 101

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(np.arange(0,len(asphericity_03)),asphericity_03)
plt.savefig('asphericity trial')

In [ ]:
#assuming the code above works to plot asphericity varying with monte carlo moves 
#can then run the system above to equillibrate over 1.5x10^7 to record the equillibrated system
#then going on to move the system again bu calcultae the asphericity every 1000 moves to calculate an average asphericity after 100 
A_035 = []
A_04 = []
A_045 = []
A_05 = []
A_055 = []
A_06 = []
A_065 = []
A_07 = []
A_075 = []
for i in range(1000,15000000):
    h_035, t_035, w_035, accepted, moved_type = metropolis_pw_2D(h_035, t_035, w_035, L, r_w_035, r_h_035, r_c_035, epsilon, Temperature, d_max, n_c_per_tail)
while len(A_035) < 100:
    for i in range(1000):
        h_035, t_035, w_035, accepted, moved_type = metropolis_pw_2D(h_035, t_035, w_035, L, r_w_035, r_h_035, r_c_035, epsilon, Temperature, d_max, n_c_per_tail)
    A_035.append(micelle_asphericity_system_full(h_035, t_035, L, 3.5 * r_h_035, 5))


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13592966302477982, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
for i in range(1000,15000000):
    h_04, t_04, w_04, accepted, moved_type = metropolis_pw_2D(h_04, t_04, w_04, L, r_w_04, r_h_04, r_c_04, epsilon, Temperature, d_max, n_c_per_tail)
while len(A_04) < 100:
    for i in range(1000):
        h_04, t_04, w_04, accepted, moved_type = metropolis_pw_2D(h_04, t_04, w_04, L, r_w_04, r_h_04, r_c_04, epsilon, Temperature, d_max, n_c_per_tail)
    A_04.append(micelle_asphericity_system_full(h_04, t_04, L, 3.5 * r_h_04, 5))

for i in range(1000,15000000):
    h_045, t_045, w_045, accepted, moved_type = metropolis_pw_2D(h_045, t_045, w_045, L, r_w_045, r_h_045, r_c_045, epsilon, Temperature, d_max, n_c_per_tail)
while len(A_045) < 100:
    for i in range(1000):
        h_045, t_045, w_045, accepted, moved_type = metropolis_pw_2D(h_045, t_045, w_045, L, r_w_045, r_h_045, r_c_045, epsilon, Temperature, d_max, n_c_per_tail)
    A_045.append(micelle_asphericity_system_full(h_045, t_045, L, 3.5 * r_h_045, 5))

In [ ]:
for i in range(1000,15000000):
    h_05, t_05, w_05, accepted, moved_type = metropolis_pw_2D(h_05, t_05, w_05, L, r_w_05, r_h_05, r_c_05, epsilon, Temperature, d_max, n_c_per_tail)
while len(A_05) < 100:
    for i in range(1000):
        h_05, t_05, w_05, accepted, moved_type = metropolis_pw_2D(h_05, t_05, w_05, L, r_w_05, r_h_05, r_c_05, epsilon, Temperature, d_max, n_c_per_tail)
    A_05.append(micelle_asphericity_system_full(h_05, t_05, L, 3.5 * r_h_05, 5))

for i in range(1000,15000000):
    h_055, t_055, w_055, accepted, moved_type = metropolis_pw_2D(h_055, t_055, w_055, L, r_w_055, r_h_055, r_c_055, epsilon, Temperature, d_max, n_c_per_tail)
while len(A_055) < 100:
    for i in range(1000):
        h_055, t_055, w_055, accepted, moved_type = metropolis_pw_2D(h_055, t_055, w_055, L, r_w_055, r_h_055, r_c_055, epsilon, Temperature, d_max, n_c_per_tail)
    A_055.append(micelle_asphericity_system_full(h_055, t_055, L, 3.5 * r_h_055, 5))

for i in range(1000,15000000):
    h_06, t_06, w_06, accepted, moved_type = metropolis_pw_2D(h_06, t_06, w_06, L, r_w_06, r_h_06, r_c_06, epsilon, Temperature, d_max, n_c_per_tail)
while len(A_06) < 100:
    for i in range(1000):
        h_06, t_06, w_06, accepted, moved_type = metropolis_pw_2D(h_06, t_06, w_06, L, r_w_06, r_h_06, r_c_06, epsilon, Temperature, d_max, n_c_per_tail)
    A_06.append(micelle_asphericity_system_full(h_06, t_06, L, 3.5 * r_h_06, 5))

    

In [ ]:
for i in range(1000,15000000):
    h_065, t_065, w_065, accepted, moved_type = metropolis_pw_2D(h_065, t_065, w_065, L, r_w_065, r_h_065, r_c_065, epsilon, Temperature, d_max, n_c_per_tail)
while len(A_065) < 100:
    for i in range(1000):
        h_065, t_065, w_065, accepted, moved_type = metropolis_pw_2D(h_065, t_065, w_065, L, r_w_065, r_h_065, r_c_065, epsilon, Temperature, d_max, n_c_per_tail)
    A_065.append(micelle_asphericity_system_full(h_065, t_065, L, 3.5 * r_h_065, 5))


for i in range(1000,15000000):
    h_07, t_07, w_07, accepted, moved_type = metropolis_pw_2D(h_07, t_07, w_07, L, r_w_07, r_h_07, r_c_07, epsilon, Temperature, d_max, n_c_per_tail)
while len(A_07) < 100:
    for i in range(1000):
        h_07, t_07, w_07, accepted, moved_type = metropolis_pw_2D(h_07, t_07, w_07, L, r_w_07, r_h_07, r_c_07, epsilon, Temperature, d_max, n_c_per_tail)
    A_07.append(micelle_asphericity_system_full(h_07, t_07, L, 3.5 * r_h_07, 5))

for i in range(1000,15000000):
    h_075, t_075, w_075, accepted, moved_type = metropolis_pw_2D(h_075, t_075, w_075, L, r_w_075, r_h_075, r_c_075, epsilon, Temperature, d_max, n_c_per_tail)
while len(A_075) < 100:
    for i in range(1000):
        h_075, t_075, w_075, accepted, moved_type = metropolis_pw_2D(h_075, t_075, w_075, L, r_w_075, r_h_075, r_c_075, epsilon, Temperature, d_max, n_c_per_tail)
    A_075.append(micelle_asphericity_system_full(h_075, t_075, L, 3.5 * r_h_075, 5))

In [ ]:
#now to calculate the standard errors on the asphericity values
A_total = [A_03, A_035, A_04, A_045, A_05, A_055, A_06, A_065, A_07, A_075]
A_averages = []
A_errors = []
for list in A_total:
    A_average = np.mean(list)
    A_averages.append(A_average)
    A_error = np.std(A_average) / np.sqrt(len(list))

#can now plot asphericity against P with errors associated to see the general trend and hope that as P increases A decreases